<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: Vitaly Radchenko (@vradchenko), [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edited by Sergey Volkov (@sevaspb). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center> Assignment #5. Fall 2018
## <center> RandomForest and Logistic Regression in credit scoring and movie reviews classification

####  Here we will develop and tune models for credit scoring and movies reviews sentiment prediction. Fill the code where needed ("#Your code is here") and answer the questions in the [web form](https://docs.google.com/forms/d/1MS3kW_bjZQAkwwlAjX9G8khj1owq1qc5NQtjzJUvKVo).

For the warm-up, solve the first task.

<font color = 'red'> **Task 1:** </font> There are 7 jurors in the courtroom. Each of them individually can correctly determine whether the defendant is guilty or not with 80% probability. How likely is the jury will make a correct verdict jointly if the decision is made by majority voting?

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q1__*

<font color = 'red'> **Answer options:** </font>
- 20.97%
- 80.00%
- 83.70%
- 96.66%


In [3]:
# 1
def f(x):
    return np.math.factorial(x)

p = 0.8
jurors = 7
m = 7 // 2 + 1
u = 0 

for i in range(m, jurors+1):
    c = f(jurors) / (f(i) * f(jurors - i))
    u += c * p ** i * (1 - p) ** (jurors - i)
print(u)

0.9666560000000001


Now let's move directly to machine learning.

#### The [dataset](https://github.com/Yorko/mlcourse.ai/tree/master/data/credit_scoring_sample.csv) looks like this:

##### Target variable
* SeriousDlqin2yrs - the person had long delays in payments during 2 years; binary variable

##### Features
* age - Age of the loan borrower (number of full years); type - integer
* NumberOfTime30-59DaysPastDueNotWorse - the number of times a person has had a delay in repaying other loans more than 30-59 days (but not more) during last two years; type - integer
* DebtRatio - monthly payments (loans, alimony, etc.) divided by aggregate monthly income, percentage; float type
* MonthlyIncome - monthly income in dollars; float type
* NumberOfTimes90DaysLate - the number of times a person has had a delay in repaying other loans for more than 90 days; type - integer
* NumberOfTime60-89DaysPastDueNotWorse - the number of times a person has had a delay in repaying other loans more than 60-89 days (but not more) in the last two years; type - integer
* NumberOfDependents - number of people in the family of the borrower; type - integer

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Let us implement a function that will replace the NaN values by the median in each column of the table.

In [4]:
def impute_nan_with_median(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table   

Reading the data:

In [5]:
data = pd.read_csv('data/credit_scoring_sample.csv', sep=";")
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


View data types of the features:

In [6]:
data.dtypes

SeriousDlqin2yrs                          int64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
NumberOfTimes90DaysLate                   int64
NumberOfTime60-89DaysPastDueNotWorse      int64
MonthlyIncome                           float64
NumberOfDependents                      float64
dtype: object

Look at the distribution of classes in target:

Distribution of target:


0    0.777511
1    0.222489
Name: SeriousDlqin2yrs, dtype: float64

In [7]:
ax = data['SeriousDlqin2yrs'].hist(orientation='horizontal', color='red')
ax.set_xlabel("number_of_observations")
ax.set_ylabel("unique_value")
ax.set_title("Target distribution")

print('Distribution of target:')
data['SeriousDlqin2yrs'].value_counts() / data.shape[0]

Select all the features and drop the target:

In [8]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

['age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'MonthlyIncome',
 'NumberOfDependents']

We apply a function that replaces all values of NaN by the median value of the corresponding column.

In [9]:
table = impute_nan_with_median(data)

Split the target and features - now we get a training sample.

In [10]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

# Bootstrap

**<font color = 'red'> Task 2. </font>** Make an interval estimate based on the bootstrap of the average income (MonthlyIncome)  of customers who had overdue loan payments, and of those who paid in time, make 90% confidence interval. Find the difference between the lower limit of the derived interval for those who paid in time and the upper limit for those who are overdue.
So, you are asked to build 90% intervals for the income of "good" customers $ [good\_income\_lower, good\_income\_upper] $ and for "bad" - $ [bad\_income\_lower, bad\_income\_upper] $ and find the difference $ good\_income\_lower - bad\_income\_upper $.

Use the example from the [article](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-5-ensembles-of-algorithms-and-random-forest-8e05246cbba7). Set `np.random.seed (17)`. Round the answer to the integer value.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q2__*

**<font color='red'>Answer options:</font>**
- 344
- 424
- 584
- 654

In [11]:
# 2
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
    

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries
    
    
np.random.seed(17)

stable = data[data.SeriousDlqin2yrs == 0 ]['MonthlyIncome'].values
debtors = data[data.SeriousDlqin2yrs == 1]['MonthlyIncome'].values

stable_mean_scores = [np.mean(sample) for sample in get_bootstrap_samples(stable, 1_000)]
debtors_mean_scores = [np.mean(sample) for sample in get_bootstrap_samples(debtors, 1_000)]
print(stat_intervals(stable_mean_scores, .1)[0] - stat_intervals(debtors_mean_scores, .1)[1])

651.9967484053386


# Decision tree, hyperparameter tuning

One of the main performance metrics of a model is the area under the ROC curve. The ROC-AUC values lay between 0 and 1. The closer the value of ROC-AUC to 1, the better the classification is done.

Find the values of `DecisionTreeClassifier` hyperparameters using the` GridSearchCV`, which maximize the area under the ROC curve.

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

Use the `DecisionTreeClassifier` class to create a decision tree. Due to the imbalance of the classes in the target, we add the balancing parameter. We also use the parameter `random_state = 17` for the reproducibility of the results.

In [13]:
dt = DecisionTreeClassifier(random_state=17, class_weight='balanced')

We will look through such values of hyperparameters:

In [14]:
max_depth_values = [5, 6, 7, 8, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

Fix cross-validation parameters: stratified, 5 partitions with shuffle, 
`random_state`.

In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

**<font color='red'>Task 3.</font>**
Run GridSearch with the ROC AUC metric using the hyperparameters from the `tree_params` dictionary. What is the maximum ROC AUC value (round up to 2 decimals)? We call cross-validation stable if the standard deviation of the metric on the cross-validation is less than 1%. Was cross-validation stable under optimal combinations of hyperparameters (i.e., providing a maximum of the mean ROC AUC value for cross-validation)?

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q3__*

**<font color='red'>Answer options:</font>**
- 0.82, no
- 0.84, no
- 0.82, yes
- 0.84, yes

In [17]:
# 3
gcv = GridSearchCV(dt, tree_params, scoring='roc_auc', n_jobs=4, cv=skf, verbose=1).fit(X, y)
# print(gcv.cv_results_)
print(gcv.best_estimator_, gcv.best_score_)

dt = DecisionTreeClassifier(random_state=17, 
                            max_depth=7, max_features=6,
                            class_weight='balanced')
np.std(cross_val_score(dt, X, y, cv=5, scoring='roc_auc'))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.9s finished


DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=7,
            max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best') 0.8203023476209635


0.006307304958968761

# Simple RandomForest implementation

**<font color='red'>Task 4.</font>**
Implement your own random forest using `DecisionTreeClassifier` with the best parameters from the previous task. There will be 10 trees, the predicted probabilities of which you need to average.

Brief specification:
 - Use the base code below
 - In the `fit` method in the loop (`i` from 0 to `n_estimators-1`), fix the seed equal to (`random_state + i`). The idea is that at each iteration there's a new value of random seed to add more "randomness", but at hte same time results are reproducible
 - After fixing the seed, select `max_features` features **without replacement**, save the list of selected feature ids in `self.feat_ids_by_tree`
 - Also make a bootstrap sample (i.e. **sampling with replacement**) of training instances. For that, resort to `np.choice` and its argument `replace`
 - Train a decision tree with specified (in a constructor) arguments `max_depth`, `max_features` and `random_state` (do not specify `class_weight`) on a corresponding subset of training data. 
 - The `fit` method returns the current instance of the class `RandomForestClassifierCustom`, that is `self`
 - In the `predict_proba` method, we need to loop through all the trees. For each prediction, obviously, we need to take only those features which we used for training the corresponding tree. The method returns predicted probabilities (`predict_proba`), averaged for all trees

Perform cross-validation. What is the average ROC AUC for cross-validation? Select the closest value.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q4__*

**<font color='red'>Answer options:</font>**
- 0.823
- 0.833
- 0.843
- 0.853

In [18]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=10, max_features=10, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        
        self.trees = []
        self.feat_ids_by_tree = []
        self.X_bootstrap = []
        self.y_bootstrap = []
        
        for i in range(0, self.n_estimators - 1):            
            self.trees.append(DecisionTreeClassifier(random_state=self.random_state + i, 
                                                     max_depth=self.max_depth, 
                                                     max_features=self.max_features
                                                    ))
            
            np.random.seed(self.random_state + i)
                        
            self.feat_ids_by_tree.append(np.random.choice(len(X.columns), self.max_features, replace=False))
            new_index = np.random.choice(data.index.values, data.shape[0], replace=True)
            
            self.X_bootstrap.append(X.iloc[new_index, self.feat_ids_by_tree[i]])
            self.y_bootstrap.append(y.iloc[new_index])
            
    def fit(self, X, y):
        for i in range(0, self.n_estimators - 1):            
            self.trees[i] = self.trees[i].fit(self.X_bootstrap[i], self.y_bootstrap[i])
            
    def predict_proba(self, X):
        proba_0 = pd.DataFrame({})
        proba_1 = pd.DataFrame({})
        total_proba = pd.DataFrame({})
        
        for i in range(0, self.n_estimators - 1):            
            proba_0[str(i)] = self.trees[i].predict_proba(X.iloc[:, self.feat_ids_by_tree[i]])[:, 0]
            proba_1[str(i)] = self.trees[i].predict_proba(X.iloc[:, self.feat_ids_by_tree[i]])[:, 1]
        total_proba['0'] = proba_0.mean(axis=1)
        total_proba['1'] = proba_1.mean(axis=1)
        return np.asarray(total_proba)
    
my_forest = RandomForestClassifierCustom(max_depth=7, max_features=6)
cross_val_score_pb = cross_val_score(my_forest, X, y, scoring='roc_auc', cv=5)
print(cross_val_score_pb)
print(np.mean(cross_val_score_pb))
print("Тут лучший результат, так как X_bootstrap созданный до fit, не позволяет кросс-валидации обрезать данные и валидация проводится 5 раз на одих и тех же, полных данных")

[0.84103132 0.8568124  0.8532263  0.84048897 0.84049525]
0.84641084755603
Тут лучший результат, так как X_bootstrap созданный до fit, не позволяет кросс-валидации обрезать данные и валидация проводится 5 раз на одих и тех же, полных данных


In [22]:
class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=10, max_features=10, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []
        self.feat_ids_by_tree = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            self.dt = DecisionTreeClassifier(random_state=self.random_state,
                                             max_depth=self.max_depth,
                                             max_features=self.max_features)
            
            np.random.seed(self.random_state + i)
            feat_ids = np.random.choice(np.arange(len(independent_columns_names)),
                                        self.max_features, replace=False)
            
            feat_ids = np.asarray(independent_columns_names)[feat_ids]
            self.feat_ids_by_tree.append(feat_ids)
            cur_bootstrap_ids = np.random.choice(np.arange(len(X)), len(X), replace=True)
            cur_X = X[self.feat_ids_by_tree[i]].values[cur_bootstrap_ids]
            cur_y = y.values[cur_bootstrap_ids]
            self.trees.append(self.dt.fit(cur_X, cur_y))

    def predict_proba(self, X):
        self.predict_proba_list = []
        for i in range(self.n_estimators):
            self.predict_proba_list.append(self.trees[i].predict_proba(X[self.feat_ids_by_tree[i]].values))
        print(np.mean(np.asarray(self.predict_proba_list), axis=0))
        return np.mean(np.asarray(self.predict_proba_list), axis=0)

my_forest = RandomForestClassifierCustom(max_depth=7, max_features=6)
cross_val_score_pb = cross_val_score(my_forest, X, y, scoring='roc_auc', cv=5)
print(cross_val_score_pb)
print(np.mean(cross_val_score_pb))

[[0.95537317 0.04462683]
 [0.94430671 0.05569329]
 [0.85811844 0.14188156]
 ...
 [0.96899195 0.03100805]
 [0.08493706 0.91506294]
 [0.31226389 0.68773611]]
[[0.87964338 0.12035662]
 [0.87409486 0.12590514]
 [0.91947299 0.08052701]
 ...
 [0.43885715 0.56114285]
 [0.84089723 0.15910277]
 [0.65008218 0.34991782]]
[[0.89903682 0.10096318]
 [0.88724034 0.11275966]
 [0.79399077 0.20600923]
 ...
 [0.61278811 0.38721189]
 [0.12033075 0.87966925]
 [0.37036176 0.62963824]]
[[0.87158671 0.12841329]
 [0.08383919 0.91616081]
 [0.39087739 0.60912261]
 ...
 [0.89003956 0.10996044]
 [0.45135553 0.54864447]
 [0.32945101 0.67054899]]
[[0.25857056 0.74142944]
 [0.87705952 0.12294048]
 [0.54401915 0.45598085]
 ...
 [0.84869958 0.15130042]
 [0.6998428  0.3001572 ]
 [0.50108933 0.49891067]]
[0.8323951  0.84849242 0.84674871 0.83135383 0.83023406]
0.8378448233802713


In [23]:
# 4
trees = []
feat_ids_by_tree = []
new_index = []
cv_roc = []

proba_0 = pd.DataFrame({})
proba_1 = pd.DataFrame({})
for i in range(0, 10 - 1):
    np.random.seed(17 + i)
    feat_ids_by_tree.append(np.random.choice(len(X.columns), 6, replace=False))
    new_index.append(np.random.choice(data.index.values, data.shape[0], replace=True))
    
    X_bootstrap = X.iloc[new_index[i], feat_ids_by_tree[i]]
    y_bootstrap = y.iloc[new_index[i]]

    dt = DecisionTreeClassifier(random_state=17+i, max_depth=7, max_features=6)
    dt.fit(X_bootstrap, y_bootstrap)
    
    X_test = X.iloc[:, feat_ids_by_tree[i]]
    proba_0[str(i)] = dt.predict_proba(X_test)[:, 0]
    proba_1[str(i)] = dt.predict_proba(X_test)[:, 1]
    
total_proba = pd.DataFrame({})
dt = DecisionTreeClassifier(random_state=17, max_depth=7, max_features=2)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
total_proba["1"] = proba_0.mean(axis=1)
total_proba["2"] = proba_1.mean(axis=1)
print(total_proba)
print(np.mean(cross_val_score(dt, total_proba, y, cv=skf, n_jobs=4, scoring="roc_auc")))

              1         2
0      0.949495  0.050505
1      0.927166  0.072834
2      0.856389  0.143611
3      0.845832  0.154168
4      0.854201  0.145799
5      0.922762  0.077238
6      0.789767  0.210233
7      0.835491  0.164509
8      0.563676  0.436324
9      0.871129  0.128871
10     0.142684  0.857316
11     0.926609  0.073391
12     0.342427  0.657573
13     0.909056  0.090944
14     0.960867  0.039133
15     0.938862  0.061138
16     0.814953  0.185047
17     0.750112  0.249888
18     0.916467  0.083533
19     0.696947  0.303053
20     0.903241  0.096759
21     0.935620  0.064380
22     0.844907  0.155093
23     0.916467  0.083533
24     0.844089  0.155911
25     0.503066  0.496934
26     0.969000  0.031000
27     0.960530  0.039470
28     0.964973  0.035027
29     0.342241  0.657759
...         ...       ...
45033  0.899291  0.100709
45034  0.938862  0.061138
45035  0.805851  0.194149
45036  0.770397  0.229603
45037  0.884458  0.115542
45038  0.618042  0.381958
45039  0.809

**<font color='red'>Task 5.</font>**
Let us compare our own implementation of a random forest with `sklearn` version of it. To do this, use `RandomForestClassifier (class_weight='balanced', random_state=17)`, specify all the same values for `max_depth` and` max_features` as before. What average value of ROC AUC on cross-validation we finally got? Select the closest value.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q5__*

**<font color='red'>Answer options:</font>**
- 0.823
- 0.833
- 0.843
- 0.853

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
# 5
rf = RandomForestClassifier(n_estimators=10, max_depth=7, max_features=6, random_state=17, class_weight='balanced')
rf.fit(X, y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
print(np.mean(cross_val_score(rf, X, y, cv=skf, n_jobs=4, scoring="roc_auc")))

0.8291436207462766


# `sklearn` RandomForest, hyperparameter tuning

**<font color='red'>Task 6.</font>** 
In the third task, we found the optimal hyperparameters for one tree. However it could be that these parameters are not optimal for an ensemble. Let's check this assumption with `GridSearchCV` `(RandomForestClassifier (class_weight='balanced', random_state = 17)` ). Now we extend the value of `max_depth` up to 15, because the trees need to be deeper in the forest (you should be aware of it from the [article](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-5-ensembles-of-algorithms-and-random-forest-8e05246cbba7)). What are the best values of hyperparameters now?

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q6__*

**<font color='red'>Answer options:</font>**
- `max_depth=8, max_features=4`
- `max_depth=9, max_features=5`
- `max_depth=10, max_features=6`
- `max_depth=11, max_features=7`

In [21]:
max_depth_values = range(5, 15)
max_features_values = [4, 5, 6, 7]
forest_params = {'max_depth': max_depth_values,
                'max_features': max_features_values}

In [ ]:
# 6
rf = RandomForestClassifier(n_estimators=10, random_state=17, class_weight='balanced')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
gcv = GridSearchCV(rf, forest_params, scoring='roc_auc', n_jobs=-1, cv=skf, verbose=1).fit(X, y)
print(gcv.best_params_, gcv.best_score_)
print("max_depth: 8, max_features: 4")

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.2s


# Logistic regression, hyperparameter tuning

**<font color='red'>Task 7.</font>**  Now let's compare our results with logistic regression (we indicate `class_weight='balanced'` and `random_state = 17`). Do a full search by the parameter `C` from a wide range of values `np.logspace(-8, 8, 17)`.
Now we will build a pipeline - first apply scaling, then train the model.

Learn about the pipelines and make cross-validation. What is the best average ROC AUC? Select the closest value.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q7__*

**<font color='red'>Answer options:</font>**
- 0.778
- 0.788
- 0.798
- 0.808

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

scaler = StandardScaler().partial_fit(X)

logit = LogisticRegression(random_state=17, class_weight='balanced')

logit_pipe = Pipeline([('scaler', scaler), ('logit', logit)])
logit_pipe_params = {'logit__C': np.logspace(-8, 8, 17)}

c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [45]:
# 7
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=17)
gscv = GridSearchCV(logit_pipe, param_grid=logit_pipe_params, scoring='roc_auc', n_jobs=4, cv=skf, verbose=1).fit(X, y)
print(gscv.best_estimator_, gscv.best_score_)

Fitting 5 folds for each of 17 candidates, totalling 85 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  61 tasks      | elapsed:    6.6s
[Parallel(n_jobs=4)]: Done  85 out of  85 | elapsed:    8.5s finished
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logit', LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=17,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))]) 0.7878249781605577


In [30]:
# 7
C_dict = {}
for C in logit_pipe_params['logit__C']:
    scaler = StandardScaler().partial_fit(X)
    logit = LogisticRegression(C=C, random_state=17, class_weight='balanced')
    
    logit_pipe_new = Pipeline([('scaler', scaler), ('logit', logit)]).fit(X, y)
    y_pred = logit_pipe_new.predict_proba(X)[:, 1]
    C_dict['C_'+str(C)] = roc_auc_score(y, y_pred)
print(C_dict)
print(max(C_dict.values()))

np.mean(cross_val_score(logit_pipe, X, y, cv=skf, n_jobs=4, verbose=1))

c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lex\appdata\local

c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
c:\users\lex\appdata\loc

c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\lex\appdata\local\programs\python

{'C_1e-08': 0.6817121731749299, 'C_1e-07': 0.6815420814066205, 'C_1e-06': 0.6799451175495244, 'C_1e-05': 0.670759561402021, 'C_0.0001': 0.6658830631419188, 'C_0.001': 0.7173047567547939, 'C_0.01': 0.7752894803903436, 'C_0.1': 0.7871233781806826, 'C_1.0': 0.788217160769447, 'C_10.0': 0.7883194464150892, 'C_100.0': 0.7883294640373935, 'C_1000.0': 0.7883302411930879, 'C_10000.0': 0.7883302554267089, 'C_100000.0': 0.7883302668136054, 'C_1000000.0': 0.788330261120157, 'C_10000000.0': 0.788330261120157, 'C_100000000.0': 0.788330261120157}
0.7883302668136054


[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished


0.7496172355708872

# Logistic regression and RandomForest on sparse features

In case of a small number of features, random forest was proved to be better than logistic regression. However, one of the main disadvantages of trees is how they work with sparse data, for example, with texts. Let's compare logistic regression and random forest in a new task.
Download dataset with reviews of movies [here](http://d.pr/f/W0HpZh). 

In [31]:
# Download data
df = pd.read_csv("data/movie_reviews_train.csv", nrows=50000)

# Split data to train and test
X_text = df["text"]
y_text = df["label"]

# Classes counts
df.label.value_counts()

       label                                               text
0          1  To an entire generation of filmgoers, it just ...
1          1  Pixar classic is one of the best kids' movies ...
2          1  Apesar de representar um imenso avanço tecnoló...
3          1  When Woody perks up in the opening scene, it's...
4          1  Introduced not one but two indelible character...
5          1  it is easy to see how virtually everything tha...
6          1  Though some of the animation seems dated compa...
7          1  Perhaps the film is meant as a pre-emptive sop...
8          1  Time has been kind to it, and future years I i...
9          1  I think I speak for all adults and kids when I...
10         1  So ingenious in concept, design and execution ...
11         1  Turns out the real magic is nothing to do with...
12         1  Its lightness of touch has not diminished, nor...
13         1  The 3-D viewing doesn't make much difference.....
14         1  It doesn't enhance the exp

1    32492
0    17508
Name: label, dtype: int64

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# Split on 3 folds
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

# In Pipeline we will modify the text and train logistic regression
classifier = Pipeline([
    ('vectorizer', CountVectorizer(max_features=100_000, ngram_range=(1, 3))),
    ('clf', LogisticRegression(random_state=17))])

**<font color='red'>Task 8.</font>** For Logistic Regression: iterate parameter `C` with values from the list [0.1, 1, 10, 100] and find the best ROC AUC in cross-validation. Select the closest answer.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q8__*

**<font color='red'>Answer options:</font>**
- 0.74
- 0.75
- 0.84
- 0.85

In [33]:
# 8
C_grid = {'clf__C': [0.1, 1, 10, 100]}
gscv = GridSearchCV(classifier, param_grid=C_grid, scoring='roc_auc', n_jobs=4, cv=skf, verbose=1).fit(X_text, y_text)
print(gscv.best_estimator_, gscv.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  1.1min finished
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
     ..., penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]) 0.858692422845382


**<font color='red'>Task 9.</font>** Now try to perform the same operation with random forest. Similarly, look over all the values and get the maximum ROC AUC. Select the closest value.

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a5_q9__*

**<font color='red'>Answer options:</font>**
- 0.74
- 0.75
- 0.84
- 0.85

In [42]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer(max_features=100000, ngram_range=(1, 3))),
    ('clf', RandomForestClassifier(random_state=17, n_jobs=4))])

min_samples_leaf = [1, 2, 3]
max_features = [0.3, 0.5, 0.7]
max_depth = [None]

print(classifier.get_params().keys())
param_grid = {'clf__min_samples_leaf': min_samples_leaf,
              'clf__max_features': max_features,
              'clf__max_depth': max_depth
             }

In [43]:
# 9
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
gscv = GridSearchCV(classifier, param_grid=param_grid, scoring='roc_auc', n_jobs=4, cv=skf, verbose=1).fit(X_text, y_text)
print(gscv.best_estimator_, gscv.best_score_)

dict_keys(['memory', 'steps', 'vectorizer', 'clf', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'clf__bootstrap', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__min_impurity_decrease', 'clf__min_impurity_split', 'clf__min_samples_leaf', 'clf__min_samples_split', 'clf__min_weight_fraction_leaf', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 39.7min finished
c:\users\lex\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
     ...timators=10, n_jobs=-1,
            oob_score=False, random_state=17, verbose=0, warm_start=False))]) 0.7472746431514513
